# Agents

In [2]:
import random

known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

## Q1. Define function description

In [3]:
get_weather_tool = {
    "type": "function",
    "name": "search",
    "description": "Search the Weather",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "What's the weather like in Germany?"
            }
        },
        "required": ["query"],
        "additionalProperties": False
    }
}

In [4]:
# !wget https://raw.githubusercontent.com/alexeygrigorev/rag-agents-workshop/refs/heads/main/chat_assistant.py

In [5]:
# !pip install minsearch

## Q2. Adding another tool

In [6]:
def set_weather(city: str, temp: float) -> None:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

In [7]:
import chat_assistant

tools = chat_assistant.Tools()
tools.add_tool(set_weather, get_weather_tool)
tools.get_tools()

[{'type': 'function',
  'name': 'search',
  'description': 'Search the Weather',
  'parameters': {'type': 'object',
   'properties': {'query': {'type': 'string',
     'description': "What's the weather like in Germany?"}},
   'required': ['query'],
   'additionalProperties': False}}]

In [9]:
from openai import OpenAI
client = OpenAI()

def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

developer_prompt = """What's the weather like in Germany?"""
chat_interface = chat_assistant.ChatInterface()

chat = chat_assistant.ChatAssistant(
    tools=tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client
)

In [ ]:
chat.run()

## Q3. Install FastMCP

In [1]:
# %pip install fastmcp
!fastmcp --version

2.10.5


## Q4. Simple MCP Server

In [3]:
# !python mcp_server.py

# Q5. Protocol 

Request: ```{"jsonrpc": "2.0", "id": 3, "method": "tools/call", "params": {"name": "get_weather", "arguments": {"city":"Berlin"}}}```

Answer: ```{"jsonrpc":"2.0","id":3,"result":{"content":[{"type":"text","text":"20.0"}],"structuredContent":{"result":20.0},"isError":false}}```

## Q6. Client

In [7]:
import mcp_client

our_mcp_client = mcp_client.MCPClient(["python", "mcp_server.py"])

our_mcp_client.start_server()
our_mcp_client.initialize()
our_mcp_client.initialized()

our_mcp_client.get_tools()
# our_mcp_client.call_tool('get_weather', {'city': 'Berlin'})

Started server with command: python mcp_server.py
Sending initialize request...
Initialize response: {'protocolVersion': '2024-11-05', 'capabilities': {'experimental': {}, 'prompts': {'listChanged': False}, 'resources': {'subscribe': False, 'listChanged': False}, 'tools': {'listChanged': True}}, 'serverInfo': {'name': 'Demo', 'version': '1.11.0'}}
Sending initialized notification...
Handshake completed successfully
Retrieving available tools...
Available tools: ['get_weather', 'set_weather']


[{'name': 'get_weather',
  'description': 'Retrieves the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to retrieve weather data.\n\nReturns:\n    float: The temperature associated with the city.',
  'inputSchema': {'properties': {'city': {'title': 'City', 'type': 'string'}},
   'required': ['city'],
   'type': 'object'},
  'outputSchema': {'properties': {'result': {'title': 'Result',
     'type': 'number'}},
   'required': ['result'],
   'title': '_WrappedResult',
   'type': 'object',
   'x-fastmcp-wrap-result': True}},
 {'name': 'set_weather',
  'description': "Sets the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to set the weather data.\n    temp (float): The temperature to associate with the city.\n\nReturns:\n    str: A confirmation string 'OK' indicating successful update.",
  'inputSchema': {'properties': {'city': {'title': 'City', 'type': 'string'},
    'temp': {'title': 'Temp